In [1]:
%matplotlib inline

from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import menpo.io as mio
from menpo.math import log_gabor
from menpo.feature import fast_dsift
from menpo.image import Image
from menpo.visualize import print_dynamic, progress_bar_str, visualize_images
from menpofit.visualize import visualize_fitting_results
from alaborticcv2015.alignment import (
    LKFitter, 
    FilteredSSD, FilteredFourierSSD, ECC, GradientImages, GradientCorrelation)
from alaborticcv2015.alignment.result import SerializableResult

# Experiment 2: YaleB Database, Previous Methods

In [2]:
db = mio.import_pickle('/Users/joan/Desktop/alaborticcv2015/data/yaleb.pkl.gz')

In [5]:
folder_path = '/Users/joan/Desktop/alaborticcv2015/results/yaleb/'

diagonal = 100
scales = (1,)
max_iters = 100

n_rep = 20
noise_std = range(2, 20, 2)

## SSD 

In [6]:
errors = np.empty((len(noise_std), len(db), len(db[0]['images']), n_rep, max_iters+1))
for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(db)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = db[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=FilteredSSD)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string2 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
mio.export_pickle(errors, folder_path + '_ssd.pkl.gz', overwrite=True)

- noise: 2,  sub: 0,  img: 9,  mean: 0.0152 - 63.4220  median: 0.0148 - 4.2245  std: 0.0045 - 132.1023 
- noise: 2,  sub: 1,  img: 9,  mean: 0.0152 - 132.7391  median: 0.0148 - 31.0424  std: 0.0045 - 230.4610 
- noise: 2,  sub: 2,  img: 9,  mean: 0.0152 - 6.6023  median: 0.0148 - 6.7386  std: 0.0045 - 4.5489 
- noise: 2,  sub: 3,  img: 9,  mean: 0.0152 - 308628.7252  median: 0.0148 - 78241.0247  std: 0.0045 - 692170.4258 
- noise: 2,  sub: 4,  img: 9,  mean: 0.0152 - 0.4898  median: 0.0148 - 0.5339  std: 0.0045 - 0.2785 
- noise: 2,  sub: 5,  img: 9,  mean: 0.0152 - 2.0437  median: 0.0148 - 0.4650  std: 0.0045 - 2.5865 
- noise: 2,  sub: 6,  img: 9,  mean: 0.0152 - 6.5439  median: 0.0148 - 0.5276  std: 0.0045 - 9.1285 
- noise: 2,  sub: 7,  img: 9,  mean: 0.0152 - 0.6802  median: 0.0148 - 0.7704  std: 0.0045 - 0.5152 
- noise: 2,  sub: 8,  img: 9,  mean: 0.0152 - 4.1929  median: 0.0148 - 3.6170  std: 0.0045 - 3.8993 
- noise: 2,  sub: 9,  img: 9,  mean: 0.0152 - 28.5112  median: 0.0148

## LogGabor 

In [7]:
def kernel_func(shape):
    k = np.fft.ifftshift(log_gabor(np.ones(shape[-2:]))[2]) 
    return k

errors = np.empty((len(noise_std), len(db), len(db[0]['images']), n_rep, max_iters+1))
for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(db)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = db[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=FilteredFourierSSD,
                          kernel_func=kernel_func)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string2 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
mio.export_pickle(errors, folder_path + '_loggabor.pkl.gz', overwrite=True)

- noise: 2,  sub: 0,  img: 9,  mean: 0.0152 - 2889.3055  median: 0.0148 - 16.5408  std: 0.0045 - 6305.6134 
- noise: 2,  sub: 1,  img: 9,  mean: 0.0152 - 15773.8590  median: 0.0148 - 5101.5296  std: 0.0045 - 26856.4236 
- noise: 2,  sub: 2,  img: 9,  mean: 0.0152 - 22.2980  median: 0.0148 - 19.8765  std: 0.0045 - 18.8886 
- noise: 2,  sub: 3,  img: 9,  mean: 0.0152 - 9228798538.8210  median: 0.0148 - 2030386420.7946  std: 0.0045 - 20865035050.1768 
- noise: 2,  sub: 4,  img: 9,  mean: 0.0152 - 0.8416  median: 0.0148 - 0.6623  std: 0.0045 - 0.6277 
- noise: 2,  sub: 5,  img: 9,  mean: 0.0152 - 12.0819  median: 0.0148 - 0.6036  std: 0.0045 - 18.1086 
- noise: 2,  sub: 6,  img: 9,  mean: 0.0152 - 61.3061  median: 0.0148 - 2.9274  std: 0.0045 - 88.6984 
- noise: 2,  sub: 7,  img: 9,  mean: 0.0152 - 1.3618  median: 0.0148 - 0.9259  std: 0.0045 - 1.4196 
- noise: 2,  sub: 8,  img: 9,  mean: 0.0152 - 37.7189  median: 0.0148 - 25.3261  std: 0.0045 - 42.7430 
- noise: 2,  sub: 9,  img: 9,  mean

## ECC 

In [ ]:
errors = np.empty((len(noise_std), len(db), len(db[0]['images']), n_rep, max_iters+1))

for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(db)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = db[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=ECC)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string2 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
mio.export_pickle(errors, folder_path + '_ecc.pkl.gz', overwrite=True)

## Gradient Images 

In [9]:
errors = np.empty((len(noise_std), len(db), len(db[0]['images']), n_rep, max_iters+1))

for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(db)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = db[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=GradientImages)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string2 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
mio.export_pickle(errors, folder_path + '_gi.pkl.gz', overwrite=True)

- noise: 2,  sub: 0,  img: 9,  mean: 0.0152 - 0.0157  median: 0.0148 - 0.0104  std: 0.0045 - 0.0077 
- noise: 2,  sub: 1,  img: 9,  mean: 0.0152 - 0.0165  median: 0.0148 - 0.0123  std: 0.0045 - 0.0116 
- noise: 2,  sub: 2,  img: 9,  mean: 0.0152 - 0.0210  median: 0.0148 - 0.0126  std: 0.0045 - 0.0157 
- noise: 2,  sub: 3,  img: 9,  mean: 0.0152 - 0.0228  median: 0.0148 - 0.0166  std: 0.0045 - 0.0165 
- noise: 2,  sub: 4,  img: 9,  mean: 0.0152 - 0.0226  median: 0.0148 - 0.0160  std: 0.0045 - 0.0140 
- noise: 2,  sub: 5,  img: 9,  mean: 0.0152 - 0.0160  median: 0.0148 - 0.0157  std: 0.0045 - 0.0079 
- noise: 2,  sub: 6,  img: 9,  mean: 0.0152 - 0.0130  median: 0.0148 - 0.0121  std: 0.0045 - 0.0060 
- noise: 2,  sub: 7,  img: 9,  mean: 0.0152 - 0.0274  median: 0.0148 - 0.0176  std: 0.0045 - 0.0223 
- noise: 2,  sub: 8,  img: 9,  mean: 0.0152 - 0.0129  median: 0.0148 - 0.0063  std: 0.0045 - 0.0160 
- noise: 2,  sub: 9,  img: 9,  mean: 0.0152 - 0.0144  median: 0.0148 - 0.0134  std: 0.0045 

## Gradient Correlation

In [10]:
errors = np.empty((len(noise_std), len(db), len(db[0]['images']), n_rep, max_iters+1))

for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(db)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = db[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=GradientCorrelation)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string2 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
mio.export_pickle(errors, folder_path + 'yaleb_gc.pkl.gz', overwrite=True)

- noise: 2,  sub: 0,  img: 9,  mean: 0.0152 - 0.0063  median: 0.0148 - 0.0061  std: 0.0045 - 0.0021 
- noise: 2,  sub: 1,  img: 9,  mean: 0.0152 - 0.0140  median: 0.0148 - 0.0136  std: 0.0045 - 0.0048 
- noise: 2,  sub: 2,  img: 9,  mean: 0.0152 - 0.0134  median: 0.0148 - 0.0118  std: 0.0045 - 0.0076 
- noise: 2,  sub: 3,  img: 9,  mean: 0.0152 - 0.0118  median: 0.0148 - 0.0114  std: 0.0045 - 0.0022 
- noise: 2,  sub: 4,  img: 9,  mean: 0.0152 - 0.0441  median: 0.0148 - 0.0221  std: 0.0045 - 0.0390 
- noise: 2,  sub: 5,  img: 9,  mean: 0.0152 - 0.0116  median: 0.0148 - 0.0111  std: 0.0045 - 0.0037 
- noise: 2,  sub: 6,  img: 9,  mean: 0.0152 - 0.0154  median: 0.0148 - 0.0120  std: 0.0045 - 0.0137 
- noise: 2,  sub: 7,  img: 9,  mean: 0.0152 - 0.0147  median: 0.0148 - 0.0112  std: 0.0045 - 0.0112 
- noise: 2,  sub: 8,  img: 9,  mean: 0.0152 - 0.0055  median: 0.0148 - 0.0053  std: 0.0045 - 0.0018 
- noise: 2,  sub: 9,  img: 9,  mean: 0.0152 - 0.0112  median: 0.0148 - 0.0103  std: 0.0045 

## dSift 

In [ ]:
errors = np.empty((len(noise_std), len(db), len(db[0]['images']), n_rep, max_iters+1))

for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(db)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = db[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          features=fast_dsift,
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=FilteredSSD)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string1 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
mio.export_pickle(errors, folder_path + 'yaleb_dsift.pkl.gz', overwrite=True)

- noise: 2,  sub: 0,  mean: 0.0152 - 0.0332  median: 0.0148 - 0.0293  std: 0.0045 - 0.0231 
- noise: 2,  sub: 1,  mean: 0.0152 - 0.0431  median: 0.0148 - 0.0347  std: 0.0045 - 0.0234 
- noise: 2,  sub: 2,  mean: 0.0152 - 0.0296  median: 0.0148 - 0.0239  std: 0.0045 - 0.0216 
- noise: 2,  sub: 3,  mean: 0.0152 - 0.0373  median: 0.0148 - 0.0343  std: 0.0045 - 0.0139 
- noise: 2,  sub: 4,  mean: 0.0152 - 0.0397  median: 0.0148 - 0.0359  std: 0.0045 - 0.0162 
- noise: 2,  sub: 5,  mean: 0.0152 - 0.0312  median: 0.0148 - 0.0218  std: 0.0045 - 0.0265 
- noise: 2,  sub: 6,  mean: 0.0152 - 0.0274  median: 0.0148 - 0.0246  std: 0.0045 - 0.0085 
- noise: 2,  sub: 7,  mean: 0.0152 - 0.0250  median: 0.0148 - 0.0245  std: 0.0045 - 0.0101 
- noise: 2,  sub: 8,  mean: 0.0152 - 0.0144  median: 0.0148 - 0.0126  std: 0.0045 - 0.0074 
- noise: 2,  sub: 9,  mean: 0.0152 - 0.0419  median: 0.0148 - 0.0384  std: 0.0045 - 0.0245 
- noise: 4,  sub: 0,  mean: 0.0304 - 0.0331  median: 0.0296 - 0.0302  std: 0.009